In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_validate, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, accuracy_score, f1_score, confusion_matrix

import xgboost as xgb
from xgboost import XGBClassifier
import lightgbm as lgb
from lightgbm import LGBMClassifier

import warnings
warnings.filterwarnings("ignore")

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

![botton_logo](https://upload.wikimedia.org/wikipedia/commons/c/ca/Titanic_Starboard_View_1912.gif)
Notebook por [Eloízio Dantas](https://www.linkedin.com/in/eloiziohmdantas/). Esta é a lendária competição [Titanic ML do Kaggle](https://www.kaggle.com/competitions/titanic).

O naufrágio do Titanic é um dos naufrágios mais infames da história. Em 15 de abril de 1912, durante sua viagem inaugural, o amplamente considerado “inafundável” RMS Titanic afundou após colidir com um iceberg. Infelizmente, não havia botes salva-vidas suficientes para todos a bordo, resultando na morte de 1.502 dos 2.224 passageiros e tripulantes. Embora houvesse algum elemento de sorte envolvido na sobrevivência, parece que alguns grupos de pessoas eram mais propensos a sobreviver do que outros. Neste desafio, pedimos que você construa um modelo preditivo que responda à pergunta: “que tipo de pessoa tem mais probabilidade de sobreviver?” usando dados de passageiros (ou seja, nome, idade, sexo, classe socioeconômica, etc).

# 1. Carregando os dados
Inicialmente é preciso apenas que os dados sejam carregados de forma correta.

In [2]:
df_train = pd.read_csv('/kaggle/input/titanic/train.csv')
df_test = pd.read_csv('/kaggle/input/titanic/test.csv')

display(df_train.head())
display(df_test.head())

Desta feita, podemos passar para etapa do pré-processamento para limpeza, integração, redução e transformação. Decidi escrever esse notebook em português, de modo que para melhor entendimento um dicionário de dados será de grande utilidade para os demais  colegas.

| Variável | Descrição | Chave |
|----------|-----------|-------|
| PassengerId | Identificador do passageiro  | |
| Survived | Sobrevivente | 0 = Não, 1 = Sim |
| Pclass | Classe do bilhete | 1 = Superior, 2 = Médio, 3 = Inferior |
| Name | Nome do Passageiro | |
| Sex | Sexo | |
| Age | Idade em anos | |
| SibSp | Nº de irmãos/cônjuges a bordo do Titanic |
| Parch | Nº de pais/filhos a bordo do Titanic |
| Ticket | Nº do tícket |
| Fare | Tarifa do passageiro |  |
| Cabin | Número da Cabine |  |
| Embarked | Porto de embarcação | C = Cherbourg, Q = Queenstown, S = Southampton |

# 2. Pré-processamento

In [3]:
# Valores faltantes
pd.DataFrame(data = [df_train.isna().sum()/df_train.shape[0]*100, df_test.isna().sum()/df_test.shape[0]*100], index=["Train Null (%)", "Test Null (%)"])

O código acima verifica valores faltantes em porcentagem. `NaN` é o dados que nosso modelo irá prever por isso ausente. Existem cerca de 20% de dados faltantes na idade, 0,2% sobre fare no dado de test e onde foi embarcado no dado de treino. O maior número de dados faltantes está em Cabin, cerca de 78%. Precisaremos analisar esses dados melhor para decidir se dropamos a coluna ou fazemos alguma imputação.

In [4]:
# Dados duplicados
print(f'Nos dados de Treino(df_train) {df_train.duplicated().sum()} dados duplicados.')
print(f'Nos dados de Teste(df_test) {df_test.duplicated().sum()} dados duplicados')

In [5]:
# Chegando informações dos dataset's
df_train.info()

Temos 891 registros em 12 variáveis. Distribuídos da seguinte forma:

| Tipo | Subtipo | Variáveis |
|---|---|---|
| Numeral | Discreta | <ul><li>SibSp</li><li>Parch</li></ul> |
|         | Continuo | <ul><li>Age</li><li>Fare</li></ul> |
| Categórica | Nominal | <ul><li>Sex</li><li>Embarked</li><li>Survived(Alvo)</li></ul> |
|            | Ordinal | <ul><li>Pclass</li></ul> |
| Outros | Texto | <ul><li>Ticket</li><li>Name</li></ul> |
|        | ID | <ul><li>PassengerId</li></ul>

# 3. Análise exploratória de dados

Seguindo a lógica da distribuição dos dados vamos explorar conforme os tipos dos dados. 

In [6]:
num_var = ['SibSp', 'Parch', 'Age', 'Fare']
cat_var = ['Sex', 'Embarked', 'Pclass']
alvo = 'Survived'

Construir algumas funções para a plotagem de gráficos para ajudar a interpretar os dados.

In [7]:
# Distribuição numérica histograma e boxplot
def num_dist(data, var):
    fig, ax = plt.subplots(1, 2, figsize=(12, 4))
    
    sns.histplot(data=data, x=var, kde=True, ax=ax[0])
    sns.boxplot(data=data, x=var, ax=ax[1])
    
    ax[0].set_title(f'{var} distribuição do histograma')
    ax[1].set_title(f'{var} distribuição do boxplot')
                    
    plt.show()

In [8]:
# Distribuição categorica pizza e barra
def cat_dist(data, var):
    fig, ax = plt.subplots(1, 2, figsize=(12, 4))

    df_train[var].value_counts().plot(kind='pie', colors=['#e86262', '#6262e8'], explode=[0.05 for x in data[var].dropna().unique()], autopct='%1.1f%%', ax=ax[0], shadow=True)
    
    ax[0].set_title(f'{var} em Pizza')
    ax[0].set_ylabel('')

    count = sns.countplot(x=var, data=df_train, ax=ax[1], palette=['r', 'b', 'g'], alpha=0.7)
    for bar in count.patches:
        count.annotate(format(bar.get_height()),
            (bar.get_x() + bar.get_width() / 2,
            bar.get_height()), ha='center', va='center',
            size=11, xytext=(0, 8),
            textcoords='offset points')
    ax[1].set_title(f'{var} em Barras')
    
    plt.show()

Vamos checar a distribuição do alvo:

In [9]:
cat_dist(df_train, alvo)

É, o acidente do RMS Titanic foi bem mortal. Lembrando o dicionário de dados o valor 0 significa que não sobreviveu, infelizmente 61.6% não sobreviveram, ou seja,total de 891 passageiros nos dados de treinamento apenas 342 sobreviveram. Então, o que os levou a sobreviver ao acidente?

In [10]:
# Verificando a distribuição do preditor
df_train[num_var].describe()

In [11]:
for var in num_var:
    num_dist(df_train, var)

A maioria dos passageiros viajava sozinho sem suas famílias. A idade dos passageiros também variáva entre 0,42 a 80 anos, com média de 29,7 anos. As tarifas variava bastante, o curioso é que alguns passageiros não precisaram pagar* (será que erá tripulante?).

Como podemos ver, todas as variáveis acima possuem outliers. A variável idade parece ter uma distribuição quase normal, mas existem alguns outliers que fazem com que a distribuição seja enviesada para a direita, já as demais variáveis possuem um distribuição assimétrica à esquerda.

*Boxplot da classe, seria melhor plotar um outra forma.*

In [12]:
for var in cat_var:
    cat_dist(df_train, var)

64,8% dos passageiros deste conjunto de treinamento são do sexo masculino, enquanto os 35,2% restantes são do sexo feminino. Mais de 70% destes passageiros embarcaram no porto Southampton (S). Muito poucos passageiros embarcaram no porto Queenstown (Q), que é apenas 8,7%, enquanto o restante embarcou no porto Cherbourg (C). A maioria dos passageiros tem 3ª classe de passagem, enquanto o número de passageiros que têm 1ª e 2ª classe de passagem é quase igual.

**Entendendo a distribuição das variáveis, vejamos se construímos um perfil de Sobreviventes e Não sobreviventes**.

In [13]:
fig, ax = plt.subplots(2, 4, figsize=(20, 10))
ax = ax.flatten()

for i, var in enumerate(num_var+cat_var):
    if i < 4:
        sns.histplot(data=df_train, x=var, hue=alvo, kde=True, ax=ax[i])
    else:
        sns.countplot(data=df_train, x=var, hue=alvo, ax=ax[i])
    
    ax[i].set_title(f'{var}: Sobreviventes e Não sobreviventes')
    
plt.subplots_adjust(hspace=0.5)
plt.show()

Os dois primeiros quadros (superior a esquerda) destaca aqueles que realizavam a viagem em família e, em ambos os casos, estes passageiros tiveram melhor chance de sobreviver. Outro grupo de sobreviventes foram os das crianças (<= 10 anos). As mulheres sobreviveram em sua maioria, enquanto a maioria dos homens morreram. Por algum motivo qualquer aqueles que embarcaram no porto de Cherbourg(C). Contudo, um dos elementos que mais influenciaram foi a Classe do Bilhete, onde as 1ª e 2ª Classe tiram uma change muito próxima de sobrevivência, mas para a 3ª classe era praticamente uma sentença de morte.

Vamos tentar entender melhor a relação entre algumas variáveis.

In [14]:
sns.violinplot(data=df_train, x='Sex', y='Age', hue='Survived', split=True)
plt.show()

A taxa de sobrevivênca dos meninos é ligeiramente melhor que das meninas. Já entre os idosos, as mulheres tiveram melhor chance que os homens. 

E entre parentes?

In [15]:
fig, ax = plt.subplots(1, 2, figsize=(12, 4))

for i, var in enumerate(['SibSp', 'Parch']):
    surv = sns.barplot(data=df_train, x=var, y=alvo, ax=ax[i], ci=None)
    for bar in surv.patches:
        surv.annotate(format('{:.3f}'.format(bar.get_height())),
            (bar.get_x() + bar.get_width() / 2,
            bar.get_height()), ha='center', va='center',
            size=11, xytext=(0, 8),
            textcoords='offset points')
        
    ax[i].set_title(f'{var} Taxa de sobrevivência')

O gráfico acima mostra que passageiros com pequeno número de familiares tendem a sobreviver. Enquanto isso, existem apenas cerca de 34% dos passageiros solitários (SibSpb e Parch = 0) que sobreviveram. Infelizmente, 0% dos passageiros com SibSp > 4 sobreviveram, e quase nenhum passageiro com Parch > 3 sobreviveu.

In [16]:
fig, ax = plt.subplots(1, 3, figsize=(20, 6))

for i, pc in enumerate(sorted(df_train['Pclass'].unique())):
    sns.histplot(data=df_train[df_train['Pclass']==pc], x='Fare', hue=alvo, kde=True, ax=ax[i])
    ax[i].set_title(f'Fare in Pclass {pc} Survival Rate')

Os passageiros das 1ª classe tiveram melhor chance de sobrevivência que da 2ª classe, que tiveram ainda melhores chances que da 3ª classe. Porém, houveram mortos em todas as classes.

# 4. Limpeza e Organização dos Dados

## 4.1. PassengerId

In [17]:
df_train['PassengerId']

Essa `PassengerId` possui apenas valores dos números de identidade dos passageiros, algo que não pode ser categorizado. Melhor remover essa variável.

In [18]:
df_train.drop('PassengerId', axis=1, inplace=True)
df_test.drop('PassengerId', axis=1, inplace=True)

## 4.2. Name

In [19]:
df_train["Name"]

Dados como nomes podem carregar muitas informações, como neste caso os pronomes de tratamentos (Mr., Mrs., Miss, etc.). Vamos extrair o valor da string seguido pelo ponto (`.`).

In [20]:
df_train['Pronome'] = df_train['Name'].str.extract('([A-Za-z]+)\.')
df_test['Pronome'] = df_test['Name'].str.extract('([A-Za-z]+)\.')

df_train['Pronome'].value_counts()

Para melhor resultado na previsão melhor reduzir a poucas classes. Vamos reduzir a apenas 5 classes.

In [21]:
# Função para ajudar a extrar pronomes
def extrair_titulo(Pronome):
    if Pronome in ['Ms', 'Mile', 'Miss']:
        return 'Miss'
    elif Pronome in ['Mme', 'Mrs']:
        return 'Mrs'
    elif Pronome == 'Mr':
        return 'Mr'
    elif Pronome == 'Master':
        return 'Master'
    else:
        return 'Other'
    
df_train['Pronome'] = df_train['Pronome'].map(extrair_titulo)
df_test['Pronome'] = df_test['Pronome'].map(extrair_titulo)

df_train['Pronome'].value_counts()

Nomes não serão mais necessários.

In [22]:
df_train.drop('Name', axis=1, inplace=True)
df_test.drop('Name', axis=1, inplace=True)

## 4.3. Ticket

In [23]:
df_train['Ticket']

Essa variável também contém apenas o valor único do bilhete para cada passageiro, e não há padrão que possamos extrair. Então, vamos apenas remover essa variável

In [24]:
df_train.drop('Ticket', axis=1, inplace=True)
df_test.drop('Ticket', axis=1, inplace=True)

## 4.4. Cabin

Essa variável tem muitos valores ausentes nos dados de treinamento e teste (mais de 77%), portanto, essa variável está perdendo muitas informações. Remover essa variável é ação adequada.df_train.drop("Cabin", axis=1, inplace=True)
df_test.drop("Cabin", axis=1, inplace=True)

In [25]:
df_train.drop('Cabin', axis=1, inplace=True)
df_test.drop('Cabin', axis=1, inplace=True)

## 4.5. Age

A variável idade tem um valor ausente de 19,87% nos dados de treinamento e 20,57% nos dados de teste. Esse número não é muito, então vamos tentar fazer a imputação de dados nessa variável. Então, como vamos fazer isso?

Normalmente, podemos apenas preencher os valores ausentes usando valores médios ou medianos. Mas o problema é que esse conjunto de dados contém muitos passageiros com idades diferentes. Nós simplesmente não podemos atribuir uma criança de 4 anos ou um homem de 60 anos com a idade média de 29 anos. Primeiramente, vamos verificar a correlação da variável idade com outras variáveis.

In [26]:
df_train.corr()['Age'].sort_values(ascending=False)

A correlação deles não parece tão boa. Agora vamos tentar verificar uma variável categórica que possa classificar a idade dos passageiros, que é a variável Pronome.

In [27]:
sns.violinplot(data=df_train, x='Pronome', y='Age')
plt.show()

Isso parece muito bom. Podemos usar a variável Pronome para classificar a idade dos passageiros. Vamos verificar a idade média do passageiro com base em seu Pronome.

In [28]:
df_train.groupby('Pronome')['Age'].mean()

In [29]:
data = [df_train, df_test]
for df in data:
    df.loc[(df["Age"].isnull()) & (df['Pronome']=='Master'), 'Age'] = 5
    df.loc[(df["Age"].isnull()) & (df['Pronome']=='Miss'), 'Age'] = 22
    df.loc[(df["Age"].isnull()) & (df['Pronome']=='Mr'), 'Age'] = 32
    df.loc[(df["Age"].isnull()) & (df['Pronome']=='Mrs'), 'Age'] = 36
    df.loc[(df["Age"].isnull()) & (df['Pronome']=='Other'), 'Age'] = 44

## 4.6. Fare

Sabemos que a variável Fare nos dados de teste tem valor ausente, então tentaremos imputar o valor dessa variável com base nos dados do trem. Vamos verificar a correlação da variável Fare com outras variáveis.

In [30]:
df_train.corr()['Fare'].sort_values(ascending=False)

Como podemos ver, a variável `Pclass` tem uma correlação negativa relativamente forte com a variável Fare. Assim, usaremos o valor médio de Fare com base na variável `Pclass` para preencher os valores ausentes.

In [31]:
df_test[df_test['Fare'].isna()]

O passageiro com tarifa em falta tem Pclass = 3

In [32]:
df_test.Fare.fillna(df_train.groupby('Pclass').mean()['Fare'][3], inplace=True)

## 4.7. SibSp and Parch

Podemos criar uma nova variável que mostre o número de famílias que acompanham sua viagem somando os valores das variáveis SibSp e Parch. E a partir dessas variáveis, também podemos criar uma variável que indica se o passageiro está sozinho ou não.

In [33]:
data = [df_train, df_test]
for df in data:
    df['Relatives'] = df['SibSp'] + df['Parch']
    df.loc[df['Relatives'] > 0, 'Alone'] = 1
    df.loc[df['Relatives'] == 0, 'Alone'] = 0

Podemos remover as variáveis ​​SIbSp e Parch, pois não precisamos mais delas

In [34]:
df_train.drop(['SibSp', 'Parch'], axis=1, inplace=True)
df_test.drop(['SibSp', 'Parch'], axis=1, inplace=True)

## 4.8. Codificação de Variável Categórica

Algumas variáveis como Sexo, Embarcado e Título são categóricas, então precisamos codificá-las primeiro para que possam ser usadas em modelos de aprendizado de máquina.

In [35]:
df_train = pd.get_dummies(df_train, prefix=['Sex', 'Embarked', 'Pronomes'])
df_test = pd.get_dummies(df_test, prefix=['Sex', 'Embarked', 'Pronomes'])

# 5. Construindo Modelos

## 5.1. Dividindo o conjunto de dados

In [36]:
X_train = df_train.drop('Survived', axis=1)
y_train = df_train.Survived

X_test = df_test.copy()

## 5.2. Dimensionamento de recursos

In [37]:
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

## 5.3. Construindo Modelo de Classificação

### 5.3.1. Escolhendo o melhor algorítmo

O próximo passo é escolher o melhor algoritmo que usaremos para prever os dados de teste. Tentaremos aplicar vários algoritmos aos dados de treinamento usando validação cruzada com um total de 10 dobras. Além da precisão, também usaremos a pontuação f1 para avaliar o desempenho do modelo, pois temos um conjunto de dados de desequilíbrio.

In [38]:
classifiers = {
    "KNN": KNeighborsClassifier(), 
    "LR": LogisticRegression(max_iter=1000), 
    "DT": DecisionTreeClassifier(),
    "RF": RandomForestClassifier(),
    "SVM": SVC(),
    "MLP": MLPClassifier(max_iter=1000),
    "XGB": XGBClassifier(),
    "LGBM": LGBMClassifier()
}

In [39]:
results = pd.DataFrame(columns=["Classifier", "Avg_Accuracy", "Avg_F1_Score"])
for name, clf in classifiers.items():
    model = clf
    cv_results = cross_validate(
        model, X_train_scaled, y_train, cv=10,
        scoring=(['accuracy', 'f1'])
    )

    results = results.append({
        "Classifier": name,
        "Avg_Accuracy": cv_results['test_accuracy'].mean(),
        "Avg_F1_Score": cv_results['test_f1'].mean()
    }, ignore_index=True)
    
results["Avg_Overall"] = (results["Avg_Accuracy"] + results["Avg_F1_Score"]) / 2
results = results.sort_values("Avg_Overall", ascending=False)
results

In [40]:
plt.figure(figsize=(12, 6))
sns.barplot(data=results, x="Avg_Overall", y="Classifier")
plt.title("Average Overall CV Score")
plt.show()

Parece que a regressão logística é o nosso melhor modelo aqui. Usaremos esse algoritmo para prever os dados de teste. Mas antes disso, vamos ajustar os hiperparâmetros neste algoritmo usando a validação cruzada de pesquisa de grade.

### 5.3.2. Ajuste de hiperparâmetro

In [41]:
lr = LogisticRegression()
params = {
    "penalty": ("l1", "l2", "elasticnet"),
    "tol": (0.1, 0.01, 0.001, 0.0001),
    "C": (10.0, 1.0, 0.1, 0.01)
}
clf = GridSearchCV(lr, params, cv=10)
clf.fit(X_train_scaled, y_train)
print("Best hyperparameter:", clf.best_params_)

In [42]:
y_pred = clf.predict(X_train_scaled)
print(f'Train Accuracy: {accuracy_score(y_train, y_pred)}')
print(f'Train F1-Score: {f1_score(y_train, y_pred)}')
sns.heatmap(confusion_matrix(y_train, y_pred), fmt='.3g', annot=True, cmap='summer_r')
plt.show()

In [43]:
print(classification_report(y_train, y_pred))

### 5.3.2. Enviar previsão de teste

In [45]:
y_pred = clf.predict(X_test_scaled)

submission = pd.read_csv('../input/titanic/gender_submission.csv')
submission['Survived'] = y_pred
submission.to_csv('submission.csv', index=False)